In [ ]:
# ==============================================================================
# 1. GEREKLİ KURULUMLAR
# ==============================================================================
print("Eski önbellek (cache) temizleniyor...")
!rm -rf /root/.cache/huggingface/

print("Gerekli kütüphaneler kuruluyor...")
!pip install flask pyngrok transformers accelerate bitsandbytes huggingface_hub torch --quiet
!apt-get install -y tshark

# ==============================================================================
# 2. GEREKLİ MODÜLLER VE GİRİŞ İŞLEMLERİ
# ==============================================================================
from flask import Flask, request, jsonify
from pyngrok import ngrok
from google.colab import userdata
from huggingface_hub import login
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import subprocess
import os
import re
import threading
import time
import json # JSON çıktısını parse etmek için

# Token'ları al (HF_TOKEN, Llama-3 erişimi için zorunludur)
try:
    hf_token = userdata.get('HF_TOKEN')
    if hf_token:
        login(token=hf_token)
        print("Hugging Face'e başarılı bir şekilde giriş yapıldı.")
    else:
        print("Hata: HF_TOKEN bulunamadı. Llama-3 erişimi için bu zorunludur.")

    ngrok_token = userdata.get('NGROK_AUTHTOKEN')
    if ngrok_token:
        ngrok.set_auth_token(ngrok_token)
        print("Ngrok kimlik doğrulaması yapıldı.")
    else:
        print("UYARI: NGROK_AUTHTOKEN bulunamadı.")

except Exception as e:
    print(f"Token hatası: {e}")

# ==============================================================================
# 3. MODEL YÜKLEME VE GENERATE FONKSİYONU (İstenen Pcap Modeli)
# ==============================================================================
print("\nModel ve Tokenizer yükleniyor... (choihyuunmin/LLaMa-PcapLog)")
model = None
tokenizer = None
try:
    # Model ID'si, sizin istediğiniz ve erişim izni aldığınız model
    model_id = "choihyuunmin/LLaMa-PcapLog"

    tokenizer = AutoTokenizer.from_pretrained(model_id)

    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        torch_dtype="auto", # Llama modelleri için 'auto' en iyisidir
        device_map="auto"   # Modeli otomatik olarak GPU'ya yükler
    )
    print("Model (choihyuunmin/LLaMa-PcapLog) başarıyla yüklendi.")

except Exception as e:
    print(f"HATA: Model yüklenemedi: {e}")
    print("Lütfen Meta Llama-3 için erişim izninizin (gated repo) onaylandığından emin olun.")

# Generate fonksiyonu Llama modelleri için optimize edildi
def generate(prompt, **kwargs):
    """LLM'den bir yanıt üretir."""
    if not model or not tokenizer: return "Model veya Tokenizer yüklenemedi."

    model_inputs = tokenizer([prompt], return_tensors="pt").to(model.device)

    # max_new_tokens=512: JSON cevabı için yeterli alan bırakır
    generated_ids = model.generate(model_inputs.input_ids, max_new_tokens=512, **kwargs)

    # Sadece üretilen yeni token'ları ayıkla
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return response.strip()

# ==============================================================================
# 4. FLASK SUNUCUSU VE API ENDPOINT'İ (Cerrahi Engelleme Mantığı)
# ==============================================================================
app = Flask(__name__)

@app.route('/analyze', methods=['POST'])
def analyze_pcap():
    if 'pcap_file' not in request.files:
        return jsonify({"error": "İstekte 'pcap_file' adında bir dosya bulunamadı.", "blocked_ips": []}), 400
    file = request.files['pcap_file']
    if not file:
        return jsonify({"error": "Dosya seçilmedi.", "blocked_ips": []}), 400

    filepath = os.path.join("uploads", f"batch_{int(time.time())}.pcap")
    os.makedirs("uploads", exist_ok=True)
    file.save(filepath)

    try:
        # --- 1. Adım: TShark ile .pcap dosyasını analiz et ---
        try:
            print(f"Gelen dosya {filepath} TShark ile analiz ediliyor...")
            command = ["tshark", "-r", filepath, "-q", "-z", "io,phs", "-z", "conv,tcp"]
            process = subprocess.run(command, capture_output=True, text=True, check=True)
            pcap_summary = process.stdout
            print("TShark analizi tamamlandı.")
        except subprocess.CalledProcessError as e:
            return jsonify({"error": f"TShark analiz hatası: {e.stderr}", "blocked_ips": []}), 500
        except FileNotFoundError:
            return jsonify({"error": "Sunucuda 'tshark' komutu bulunamadı.", "blocked_ips": []}), 500

        # --- 2. Adım: LLM için YENİ Prompt Hazırla ---
        # İstemcinin "sadece riskli IP'yi engelle" talebine uymak için,
        # Pcap modeline SKOR sormak yerine, JSON listesi istiyoruz.
        print("LLM için JSON tabanlı prompt hazırlanıyor...")
        prompt = f"""
You are a senior cybersecurity analyst AI specialized in pcap analysis (LLaMa-PcapLog). Your task is to analyze the following TShark network traffic summary and identify malicious source IP addresses.

**TASK:**
1.  Analyze the summary. Look for signs of port scanning, C2 communication, or other anomalies.
2.  Respond with a JSON object.
3.  The JSON object *must* contain a key named "blocked_ips" which is a list of strings (source IP addresses) that you deem malicious.
4.  The JSON object *must* also contain a "reason" key explaining your decision.

**EXAMPLE RESPONSES:**
- If a port scan is detected from '1.2.3.4':
{{"blocked_ips": ["1.2.3.4"], "reason": "Port scan detected from 1.2.3.4, targeting multiple ports."}}
- If traffic is normal:
{{"blocked_ips": [], "reason": "Traffic appears normal. No malicious activity detected."}}

**Begin Analysis:**

--- NETWORK TRAFFIC SUMMARY ---
{pcap_summary}
--- ANALYSIS RESULT (JSON Only) ---
"""

        model_output = generate(prompt)
        print(f"LLM Ham Çıktısı: {model_output}")

        # --- 3. Adım: LLM Çıktısını (JSON) Parse Et ve Döndür ---
        try:
            # Modelin çıktısından sadece JSON kısmını ayıkla
            json_match = re.search(r'\{.*\}', model_output, re.DOTALL)

            if json_match:
                response_json_str = json_match.group(0)
                response_data = json.loads(response_json_str)

                # İstemcinin beklediği anahtarları doğrula
                if "blocked_ips" not in response_data:
                    response_data["blocked_ips"] = []
                if "reason" not in response_data:
                    response_data["reason"] = "Model JSON döndürdü ancak 'reason' anahtarı eksik."

                print(f"İstemciye gönderiliyor: {response_data}")
                return jsonify(response_data)

            else:
                print("Hata: Model çıktısında JSON bulunamadı.")
                return jsonify({"error": "Model çıktısında JSON bulunamadı.", "details": model_output, "blocked_ips": []}), 500

        except json.JSONDecodeError:
            print("Hata: Model geçerli bir JSON döndürmedi.")
            return jsonify({"error": "Model geçersiz JSON döndürdü.", "details": model_output, "blocked_ips": []}), 500

    except Exception as e:
        print(f"API endpoint'inde beklenmedik hata: {e}")
        return jsonify({"error": f"Sunucu içi hata: {str(e)}", "blocked_ips": []}), 500

    finally:
        # Yüklenen .pcap dosyası her durumda silinir.
        if os.path.exists(filepath):
            os.remove(filepath)
            print(f"Geçici dosya {filepath} silindi.")

# ==============================================================================
# 5. SUNUCUYU BAŞLATMA VE NGROK TÜNELİ OLUŞTURMA
# ==============================================================================
def run_app():
    app.run(host='0.0.0.0', port=5001) # Port 5001 kullanılıyor

if model: # Model başarıyla yüklendiyse sunucuyu başlat
    flask_thread = threading.Thread(target=run_app)
    flask_thread.daemon = True
    flask_thread.start()

    public_url = ngrok.connect(5001)
    print("\n================================================================================")
    print("SUNUCU HAZIR! (LLaMa-PcapLog Modeli ve Cerrahi Engelleme Aktif)")
    print(f"Raspberry Pi istemcisinde LLM_API_URL olarak bu adresi kullanın:")
    print(f"{public_url}/analyze")
    print("================================================================================")
else:
    print("\nModel yüklenemediği için sunucu başlatılmadı.")

try:
    while True: time.sleep(3600)
except KeyboardInterrupt:
    ngrok.kill()
    print("Sunucu ve ngrok tüneli kapatıldı.")